# `nb08a`: Model checking

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Now that we can simulate data and fit models, we can evaluate the quality of a model and critically assess whether it is a good fit to data. This is called model checking.

# The light speed experiment 

In 1882, Simon Newcomb conducted an experiment to measure the speed of light. He used a rotating mirror to reflect a beam of light to a distant mirror and back. The time taken for the light to travel to the distant mirror and back was measured by timing the rotation of the mirror. The experiment was repeated several times, and the times taken for the light to travel to the distant mirror and back were recorded as deviations from 24,800 nanoseconds. 

In [ ]:
# data
light_speed = np.array([28, 26, 33, 24, 34, -44, 27, 16, 40, -2, 29, 22, 24, 21, 25,
                        30, 23, 29, 31, 19, 24, 20, 36, 32, 36, 28, 25, 21, 28, 29,
                        37, 25, 28, 26, 30, 32, 36, 26, 30, 22, 36, 23, 27, 27, 28,
                        27, 31, 27, 26, 33, 26, 32, 32, 24, 39, 28, 24, 25, 32, 25,
                        29, 27, 28, 29, 16, 23]) 
# light_speed = np.array([28, 26, 33, 24, 34, 27, 16, 40, -2, 29, 22, 24, 21, 25,
#                         30, 23, 29, 31, 19, 24, 20, 36, 32, 36, 28, 25, 21, 28, 29,
#                         37, 25, 28, 26, 30, 32, 36, 26, 30, 22, 36, 23, 27, 27, 28,
#                         27, 31, 27, 26, 33, 26, 32, 32, 24, 39, 28, 24, 25, 32, 25,
#                         29, 27, 28, 29, 16, 23]) 

len(light_speed)

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)
ax.hist(light_speed, bins=20, edgecolor='black')
plt.show()

In [ ]:
# model
from scipy.stats import norm, uniform

def log_likelihood(theta, x):
    return norm.logpdf(x, loc=theta[0], scale=theta[1]).sum()

def log_prior(theta):   
    return (uniform.logpdf(theta[0], loc=-1000, scale=2000) +  # p(mu) = U[-1000, 1000]
            uniform.logpdf(theta[1], loc=0.1, scale=1000))     # p(sigma) = U[0.1, 1000]

def log_posterior(theta, x):
    return log_likelihood(theta, x) + log_prior(theta) 

In [ ]:
# posterior inference
import emcee
pos = np.array([10.0, 10.0]) + 0.1 * np.random.randn(10, 2)
nwalkers, ndim = pos.shape
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=(light_speed,))
sampler.run_mcmc(pos, 300000 // nwalkers, progress=True);

In [ ]:
fig, axes = plt.subplots(ndim, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
labels = ["mu", "sigma"]

for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:5000, :, i], "r", alpha=0.1)
    ax.plot(np.mean(samples[:5000, :, i], axis=1), "r", alpha=1.0)
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number")
plt.show()

In [ ]:
tau = sampler.get_autocorr_time()
tau

In [ ]:
import corner
thetas = sampler.get_chain(flat=True, discard=500, thin=40)
fig = corner.corner(thetas, labels=labels)

In [ ]:
print("E_p(mu|x)[mu] =", np.mean(thetas[:, 0]))
print("E_p(sigma|x)[sigma] =", np.mean(thetas[:, 1]))

In [ ]:
print("Empirical mean =", np.mean(light_speed))
print("Empirical std =", np.std(light_speed))

<div class="alert alert-success">
    
**Exercise**. Determine the 95% credibility interval for $\mu$ and $\sigma$.
    
</div>

A (highest posterior density) 95% credibility interval is the shortest interval that contains 95% of the posterior probability.

In [ ]:
def hpd(trace, mass_frac) :
    d = np.sort(np.copy(trace))
    n = len(trace)
    n_samples = np.floor(mass_frac * n).astype(int)
    int_width = d[n_samples:] - d[:n-n_samples]
    min_int = np.argmin(int_width)
    return np.array([d[min_int], d[min_int+n_samples]])

low, high = hpd(thetas[:, 0], 0.95)
low, high

In [ ]:
24800+low, 24800+high

Note: The currently accepted estimate of the speed of light under the conditions of the experiment corresponds to a measurement of 24833.02 nanoseconds.

# Posterior predictive distribution

$$p(x^\text{rep} | \{x\}) = \int p(x^\text{rep} | \theta) p(\theta | \{x\}) d\theta$$

In [ ]:
thetas = thetas[:1000]

def pp(thetas):
    return norm.rvs(loc=thetas[:, 0], scale=thetas[:, 1])

$$p(\{x^\text{rep}\} | \{x\}) = \int p(\{x^\text{rep}\} | \theta) p(\theta | \{x\}) d\theta$$

In [ ]:
def replicate(n_samples, thetas):
    x = np.zeros((n_samples, len(thetas)))
    
    for i in range(n_samples):
        x[i] = pp(thetas)
    
    return x

In [ ]:
replicas = replicate(66, thetas)

In [ ]:
replicas.shape

## Visual inspection of posterior predictives

In [ ]:
fig = plt.figure()
ax = plt.axes()
ax.hist(light_speed, bins=15, range=(-50, 50))
plt.show()

In [ ]:
fig, ax = plt.subplots(5, 5, sharex=True, sharey=True)
lower = -50
upper = np.max(replicas[:, :25])
for k in range(25):
    ax[k // 5, k % 5].hist(replicas[:, k], bins=15, range=(lower, upper))
plt.show()

## Test quantities

Statistics 101: _There is only one test!_
1. Given the data, compute a test statistic that measures some aspect of the data.
2. Compute the distribution of the test statistic under the model.
3. Compare the observed test statistic to the distribution of the test statistic under the model, usually in terms of a p-value.

Forget about statistical recipes! They have been invented when computers were not available, i.e. when Step 2 was not something we could simply do by simulation.

In [ ]:
light_speed.mean()

In [ ]:
replicas.mean(axis=0)

In [ ]:
fig = plt.figure()
ax = plt.axes()

ax.hist(replicas.mean(axis=0), bins=15)
ax.axvline(light_speed.mean(), color="r")
        
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.axes()

ax.hist(replicas.std(axis=0), bins=15)
ax.axvline(light_speed.std(), color="r")
        
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.axes()

ax.hist(replicas.min(axis=0), bins=15)
ax.axvline(light_speed.min(), color="r")
        
plt.show()

In [ ]:
np.mean(replicas.min(axis=0) >= light_speed.min())

This suggests that the normal distribution is not adequate to model the data. Its tails are too light.

## Bayesian p-values

Formally,
- $T(\{x\}, \theta)$ denotes a test quantity or discrepancy measure.
- $T(\{x\})$ denotes a test-statistic.

Classical p-values:
$$p_C = P(T(\{x^\text{rep}\}) \geq T(\{x\}) | \theta)$$

Bayesian p-values:
$$p_B = P(T(\{x^\text{rep}\}, \theta) \geq T(\{x\}, \theta) | \{ x \})$$

In [ ]:
def T(xs, theta):
    xs = np.sort(xs)
    return np.abs(xs[60] - theta) - np.abs(xs[5] - theta)  # should scatter around 0 for a symmetric distribution

In [ ]:
T_reps = []
T_x = []

for k in range(len(thetas)):
    T_reps.append(T(replicas[:, k], thetas[k, 0]))
    T_x.append(T(light_speed, thetas[k, 0]))
    
T_reps = np.array(T_reps)
T_x = np.array(T_x)

# T_reps is the test statistic for the replicas
# T_x is the test statistic for the original data 
# both are computed for the same value of theta

In [ ]:
fig = plt.figure()
ax = plt.axes()

ax.scatter(T_x, T_reps)
ax.plot([-10, 12], [-10, 12])
ax.set(xlim=(-10, 12), ylim=(-10,12), xlabel=r"$T(\{x\}, \theta)$", ylabel=r"$T(\{x^{rep}\}, \theta)$")
ax.grid()

plt.show()

In [ ]:
fig = corner.corner(np.hstack([T_x.reshape(-1, 1), T_reps.reshape(-1, 1)]), labels=[r"$T(\{x\}, \theta)$", r"$T(\{x^{rep}\}, \theta)$"])

In [ ]:
np.mean(T_reps >= T_x)

_Gelman:_ "Finding an extreme p-value and thus ‘rejecting’ a model is
never the end of an analysis; the departures of the test quantity in
question from its posterior predictive distribution will often suggest
improvements of the model or places to check the data, as in the speed
of light example. Moreover, even when the current model seems
appropriate for drawing inferences (in that no unusual deviations
between the model and the data are found), the next scientific step will
often be a more rigorous experiment incorporating additional factors,
thereby providing better data."